In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [4]:
class Scraper:
    def __init__(self, keywords, pages):
        self.keywords = keywords
        self.pages = pages

    def fetch(self, base_url):
        self.base_url = base_url

        self.params = {
            'query': self.keywords,
            'sortby': 'time',
            'page': 2
        }

        self.headers = {
            'sec-ch-ua': '"(Not(A:Brand";v="8", "Chromium";v="99", "Google Chrome";v="99"',
            'sec-ch-ua-platform': "Linux",
            'sec-fetch-site': 'same-origin',
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.35 Safari/537.36'
        }

        self.response = requests.get(self.base_url, params=self.params, headers=self.headers)

        return self.response

    def get_articles(self, response):
        article_lists = []

        for page in range(1, int(self.pages)+1):
            url = f"{self.base_url}?query={self.keywords}&sortby=time&page={page}"

            page = requests.get(url)
            soup = BeautifulSoup(page.text, "html.parser")

            articles = soup.find_all("article")

            for article in articles:
                title = article.find("h2", {"class": "title"}).get_text()
                category = article.find("span", {"class": "category"}).get_text()
                published_time = article.find("span", {"class": "date"}).get_text().split(",")[1]
                href = article.find("a")["href"]
                descript = article.find("p").get_text()
                article_lists.append({
                    "title": title, 
                    "category": category, 
                    "published_time": published_time, 
                    "href": href,
                    "descript": descript})

        self.articles = article_lists

        try:
            self.show_results()
        except Exception as e:
            print(e)
        finally:
            print()
            print( "[~] Scraping finished!")
            print(f"[~] Total Articles: {len(self.articles)}")

        return self.articles

    def save_to(self, file_format="csv"):
        '''  '''
        time_scrape = datetime.now().strftime("%m%d%Y_%H%M%S")

        df = pd.DataFrame(self.articles)

        file_name = f"result_{self.keywords}_{time_scrape}"
        if file_format == "csv":
            file_name += ".csv"
            df.to_csv(file_name, index=False)
            print(f"[~] Result saved to '{file_name}'")
        elif file_format == "excel":
            file_name += ".xlsx"
            df.to_excel(file_name, index=False)
            print(f"[~] Result saved to '{file_name}'")

    def show_results(self, row = 5):
        df = pd.DataFrame(self.articles)
        df.index += 1
        if row:
            print(df.head())
        else:
            print(df)
            
if __name__ == '__main__':
    keywords = input("[~] Keywords     : ")
    pages =    input("[~] Total Pages  : ")
    base_url = f"https://www.detik.com/search/searchall"

    scrape = Scraper(keywords, pages)
    response = scrape.fetch(base_url)
    articles = scrape.get_articles(response)

    try:
        ask =             input("[~] Do you want save the results? [y/n]: ").lower()
        if ask == 'y':
            file_format = input("[~] Save to file format? [csv/excel]           : ").lower()
            scrape.save_to(file_format=file_format)
        elif ask == 'n':
            scrape.show_results()
    except Exception as e:
        print(e)
    finally:
        print("[~] Program Finished")

[~] Keywords     : pemilihan presiden 2024
[~] Total Pages  : 2
                                               title    category  \
1  SBY Bakal ke Sumut Besok, Hadiri Konsolidasi D...  detikSumut   
2  TPN Ganjar-Mahfud Sebut Hasil Survei untuk Mob...   detikBali   
3  Kapan Debat Ketiga Capres Pemilu 2024? Ini Tan...  detikJogja   
4  Debat Pilpres Ketiga Digelar Awal 2024, Temany...   detikNews   
5  Jerry Sambuaga: Rapimnas AMPI Bulatkan Tekad D...   detikNews   

           published_time                                               href  \
1   25 Des 2023 20:00 WIB  https://www.detik.com/sumut/berita/d-7108508/s...   
2   25 Des 2023 14:32 WIB  https://www.detik.com/bali/nusra/d-7108081/tpn...   
3   25 Des 2023 13:00 WIB  https://www.detik.com/jogja/berita/d-7107962/k...   
4   25 Des 2023 10:52 WIB  https://news.detik.com/pemilu/d-7107801/debat-...   
5   25 Des 2023 08:55 WIB  https://news.detik.com/pemilu/d-7107612/jerry-...   

                                            de

In [5]:
data = pd.read_csv('result_pemilihan presiden 2024_12252023_201223.csv')
data.head()

,title,category,published_time,href,descript
0,"SBY Bakal ke Sumut Besok, Hadiri Konsolidasi D...",detikSumut,25 Des 2023 20:00 WIB,https://www.detik.com/sumut/berita/d-7108508/s...,Presiden ke-6 RI yang juga Ketua Dewan Pembina...
1,TPN Ganjar-Mahfud Sebut Hasil Survei untuk Mob...,detikBali,25 Des 2023 14:32 WIB,https://www.detik.com/bali/nusra/d-7108081/tpn...,"Wakil Ketua TPN Ganjar-Mahfud, TGB Zainul Majd..."
2,Kapan Debat Ketiga Capres Pemilu 2024? Ini Tan...,detikJogja,25 Des 2023 13:00 WIB,https://www.detik.com/jogja/berita/d-7107962/k...,Berikut informasi lengkap jadwal debat ketiga ...
3,"Debat Pilpres Ketiga Digelar Awal 2024, Temany...",detikNews,25 Des 2023 10:52 WIB,https://news.detik.com/pemilu/d-7107801/debat-...,"Usai selesaikan dua debat, KPU RI akan lanjut ..."
4,Jerry Sambuaga: Rapimnas AMPI Bulatkan Tekad D...,detikNews,25 Des 2023 08:55 WIB,https://news.detik.com/pemilu/d-7107612/jerry-...,DPP Angkatan Muda Pembaharuan Indonesia (AMPI)...


In [4]:
data.shape

(2, 5)

In [5]:
for a in data["href"]:
    print(a)

https://news.detik.com/berita/d-4890526/2021-setiap-kelurahan-di-kota-semarang-dapat-anggaran-rp-1-m
https://news.detik.com/berita/d-4453250/walkot-hendi-ajak-pkk-ubah-stigma-rumpi-dengan-tingkatkan-prestasi


In [15]:
for b in data["descript"]:
    print(b)

Akademi Kepemimpinan Digital atau Digital Leadership Academy (DLA) yang diikuti para petinggi Pemkab Mojokerto berbuah manis.
Rakitis adalah kelainan pada pertumbuhan tulang anak. Ini disebabkan beberapa faktor salah satunya adalah kurangnya vitamin D. Simak penjelasan tentang rakitis.
Stunting adalah gangguan pertumbuhan dan perkembangan pada anak yang dapat mempengaruhi kognitif dan motorik. Salah satu gejalanya yakni tubuh pendek dan kurus.
Mencegah kejahatan seksual juga berarti mencegah stunting, dan itu artinya mencegah penghancuran peradaban manusia dan bangsa.
MPASI diperlukan karena setelah berusia 6 bulan, bayi membutuhkan tambahan energi, protein, dan zat besi yang tidak bisa diperoleh dari konsumsi ASI saja.
Pelaksana harian (Plh) Wali Kota Bandung Ema Sumarna menyebutkan data masih ada 5.548 anak di Kota Bandung yang mengalami stunting.
Stunting dapat disebabkan oleh berbagai faktor, salah satunya imunitas atau daya tahan tubuh yang mengalami penurunan.
Pemerintah Kota (Pe

In [16]:
data["descript"][0]

'Akademi Kepemimpinan Digital atau Digital Leadership Academy (DLA) yang diikuti para petinggi Pemkab Mojokerto berbuah manis.'